In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
PATH = "mit-bih-arrhythmia-database-1.0.0/"     #path, 这里就是写保存的数据地址
HEADERFILE = "100.hea"     #文件格式为文本格式
ATRFILE = "100.atr"     #attributes-file 文件以二进制格式
DATAFILE = "100.dat"     #data-file

####################读取头文件######################
f = open(PATH+HEADERFILE,"r")
z = f.readline().split()
nosig, sfreq = int(z[1]),int(z[2])     #% number of signals，sample rate of data
SAMPLES2READ = 10 * sfreq     #读取的数据样本点数

dformat, gain, bitres, zerovalue, firstvalue = [], [], [], [], []
for i in range(nosig):
    z = f.readline().split()
    dformat.append(int(z[1]))     #format; here only 212 is allowed
    gain.append(int(z[2]))     #number of integers per mV
    bitres.append(int(z[3]))     #bitresolution
    zerovalue.append(int(z[4]))     #integer value of ECG zero point
    firstvalue.append(int(z[5]))     #first integer value of signal (to test for errors)
f.close()

####################读取dat文件######################
#.dat文件的数据格式读取为每行三个字节,即三个八位的二进制数字,其内容含义为
#      0000 0000  ||             0000 0000              ||  0000 0000
#sign1(L)低八位信息||左四位sign2(R),右四位sign1(L)||sign2(R)低八位信息
#将第二字节的信息处理后,后四位移至第一字节最左位即得到完整的sign1
#将第二字节的信息处理后,前四位移至第一字节最左位即得到完整的sign2.

f = open(PATH+DATAFILE, "rb")     #以二进制格式读入dat文件
b = f.read()
f.close()

A_init = np.frombuffer(b,dtype=np.uint8)      #将读入的二进制文件转化为unit8格式
A_shape0 = int(A_init.shape[0]/3)
A = A_init.reshape(A_shape0,3)[:SAMPLES2READ]     #将A转为3列矩阵

M = np.zeros((SAMPLES2READ,2))     #创建矩阵M

M2H = A[:,1]>>4     #字节向右移四位，即取字节的高四位
M1H = A[:,1]&15     #取字节的低四位

PRL = (A[:,1]&8)*(2**9)     #sign-bit   取出字节低四位中最高位，向左移九位，等于乘2^9
PRR = A[:,1]&128<<5     #sign-bit   取出字节高四位中最高位，向左移五位

M1H = M1H*(2**8)
M2H = M2H*(2**8)

M[:,0] = A[:,0]+M1H-PRL
M[:,1] = A[:,2]+M2H-PRR

if ((M[1,:]!=firstvalue).any()):
    print("inconsistency in the first bit values")

if nosig==2:
    M[:, 0] = (M[:, 0] - zerovalue[0]) / gain[0]
    M[:, 1] = (M[:, 1] - zerovalue[1]) / gain[1]
    TIME=np.linspace(0,SAMPLES2READ-1,SAMPLES2READ)/sfreq
elif nosig==1:
    M2=[]
    M[:, 0] = M[:, 0] - zerovalue[0]
    M[:, 1] = M[:, 1] - zerovalue[1]
    for i in range(M.shape[0]):
        M2.append(M[:,0][i])
        M2.append(M[:,1][i])
    M2.append(0)
    del M2[0]
    M2 = np.array(M2)/gain[0]
    TIME = np.linspace(0,2*SAMPLES2READ-1,2*SAMPLES2READ)/sfreq
else:
    print("Sorting algorithm for more than 2 signals not programmed yet!")

####################读取atr文件######################
f=open(PATH+ATRFILE,"rb")     #主要是读取ATR文件中各周期数据并在之后打印在图中
b=f.read()
f.close()

A_init=np.frombuffer(b,dtype=np.uint8)
A_shape0=int(A_init.shape[0]/2)
A=A_init.reshape(A_shape0,2)

ANNOT,ATRTIME=[],[]
i=0
while i < A.shape[0]:
    annoth=A[i,1]>>2
    if annoth==59:
        ANNOT.append(A[i+3,1]>>2)
        ATRTIME.append(A[i+2,0]+A[i+2,1]*(2**8)+A[i+1,0]*(2**16)+A[i+1,1]*(2**24))
        i+=3
    elif annoth==60:pass
    elif annoth==61:pass
    elif annoth==62:pass
    elif annoth==63:
        hilfe=(A[i,1]&3)*(2**8)+A[i,0]
        hilfe=hilfe+hilfe%2
        i+=int(hilfe/2)
    else:
        ATRTIME.append((A[i,1]&3)*(2**8)+A[i,0])
        ANNOT.append(A[i,1]>>2)
    i+=1

del ANNOT[len(ANNOT)-1]
del ATRTIME[len(ATRTIME)-1]

ATRTIME=np.array(ATRTIME)
ATRTIME=np.cumsum(ATRTIME)/sfreq

ind=np.where(ATRTIME<=TIME[-1])[0]
ATRTIMED=ATRTIME[ind]

ANNOT=np.round(ANNOT)
ANNOTD=ANNOT[ind]

NameError: name 'np' is not defined